
# Install below libraries


In [171]:
# !pip install pymysql pandas
# !pip install snowflake-connector-python
# !pip install --upgrade snowflake-connector-python
# !pip install "snowflake-connector-python[secure-local-storage,pandas]"


# import libraries

In [172]:
import pymysql
import pandas as pd
import snowflake.connector

# Connect to MariaDB

In [173]:

connection = pymysql.connect(
    host='localhost',
    user='root',
    password='21Believe21',
    db='bhagyesh_db'
)

In [174]:
# CREATE OR REPLACE TABLE employees (
#     employee_id INT,
#     first_name STRING,
#     last_name STRING,
#     department STRING,
#     salary NUMBER
# )

# INSERT INTO employees (employee_id, first_name, last_name, department, salary)
# VALUES
#     (1, 'John', 'Doe', 'IT', 60000),
#     (2, 'Jane', 'Smith', 'HR', 55000),
#     (3, 'Michael', 'Johnson', 'Finance', 70000),
#     (4, 'Emily', 'Williams', 'Marketing', 60000),
#     (5, 'Daniel', 'Brown', 'IT', 65000),
#     (6, 'Olivia', 'Jones', 'HR', 58000),
#     (7, 'David', 'Miller', 'Finance', 72000),
#     (8, 'Sophia', 'Davis', 'Marketing', 61000),
#     (9, 'James', 'Wilson', 'IT', 63000),
#     (10, 'Ava', 'Moore', 'HR', 59000);
    
# SELECT * FROM employees;

# write a query

In [175]:

 
query=""" SELECT * FROM employees;"""

# Create a dataframe

In [176]:

df = pd.read_sql(query, connection)
df

C:\Users\BHAGYESH\AppData\Local\Temp/ipykernel_23624/208755241.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,employee_id,first_name,last_name,department,salary
0,1,John,Doe,IT,60000.0
1,2,Jane,Smith,HR,55000.0
2,3,Michael,Johnson,Finance,70000.0
3,4,Emily,Williams,Marketing,60000.0
4,5,Daniel,Brown,IT,65000.0
5,6,Olivia,Jones,HR,58000.0
6,7,David,Miller,Finance,72000.0
7,8,Sophia,Davis,Marketing,61000.0
8,9,James,Wilson,IT,63000.0
9,10,Ava,Moore,HR,59000.0


# Export Employees table data to csv file

In [177]:
csv_data=df.to_csv('temp.csv',index=False)

# close connection

In [178]:
connection.close()

# Create Snowflake connection


In [179]:
snowflake_conn = snowflake.connector.connect(
    user='BHAGYESH',
    password='21Believe21',
    account='tvpxgnt-wb46719',
    warehouse='COMPUTE_WH',
    database='TESY',
    schema='PUBLIC'
)

snowflake_cursor = snowflake_conn.cursor()

# Create a stage in Snowflake

In [180]:
stage_name = 'pandas_to_snowflake_stage'
create_stage_query = f"CREATE OR REPLACE STAGE {stage_name}"
snowflake_cursor.execute(create_stage_query)


# Put the CSV data in the stage

In [181]:
put_query = f"PUT file://temp.csv @{stage_name}"
snowflake_cursor.execute(put_query, csv_data)

# Copy data from the stage into the Snowflake table

In [182]:

copy_query = f"COPY INTO employee FROM @{stage_name}/temp.csv FILE_FORMAT=(TYPE=CSV SKIP_HEADER=1)"
snowflake_cursor.execute(copy_query)


In [183]:
# Query and fetch data from Snowflake
snowflake_query = "SELECT * FROM employee"
snowflake_cursor = snowflake_conn.cursor()
snowflake_cursor = snowflake_conn.cursor()
snowflake_cursor.execute(snowflake_query)
snowflake_df = snowflake_cursor.fetch_pandas_all()

In [187]:
snowflake_df.dtypes

EMPLOYEE_ID      int8
FIRST_NAME     object
LAST_NAME      object
DEPARTMENT     object
SALARY          int32
dtype: object

# Close the cursor and connection

In [185]:
# Close Snowflake connection
snowflake_cursor.close()
snowflake_conn.close()

In [186]:


# Perform Quality Check
def perform_quality_check(df1, df2):
    if df1.shape != df2.shape:
        return "Row count mismatch"
    
#     if list(df1.columns) != list(df2.columns):
#         return "Column names mismatch"
    
    if not df1.dtypes.equals(df2.dtypes):
        return "Data types mismatch"
    
    return "Data quality check passed"

result = perform_quality_check(maria_df, snowflake_df)
print(result)

Data types mismatch
